# Cosmic shear with Einstein ring simulations

In [ ]:
# import of standard python libraries
import numpy as np
import os
import time
import corner


plotting = True
if plotting:
    import matplotlib.pyplot as plt
    from mpl_toolkits.axes_grid1 import make_axes_locatable
    %matplotlib inline

## simulation choices

In [ ]:
# import main simulation class of lenstronomy
from lenstronomy.SimulationAPI.simulations import Simulation
SimAPI = Simulation()

# data specifics
sigma_bkg = .005  #  background noise per pixel
exp_time = 500.  #  exposure time (arbitrary units, flux per pixel is in units #photons/exp_time unit)
numPix = 60  #  cutout pixel size
deltaPix = 0.05  #  pixel size in arcsec (area per pixel = deltaPix**2)
fwhm = 0.01  # full width half max of PSF
psf_type = 'gaussian'  # 'gaussian', 'pixel', 'NONE'

gamma = 2.
# lensing quantities
kwargs_shear = {'e1': 0.0, 'e2': -0.1}  # shear values to the source plane
kwargs_foreground = {'gamma1_foreground': 0.0, 'gamma2_foreground':-0.06}  # non-linear shear values to the lens plane
kwargs_spemd = {'theta_E': .66, 'gamma': gamma, 'center_x': 0.05, 'center_y': 0, 'q': .9, 'phi_G': 0.}  # parameters of the deflector lens model

# choice of source type
source_type = 'SERSIC'  # 'SERSIC' or 'SHAPELETS'

beta_model = 0.01  # choice of source size in the input simulation (units of arc seconds)
shapelet_beta = 0.01  # choice of initial shapelet scale in the reconstruction in the lens modelling
n_max_sim = 10  # choice of polynomial order in the shapelet reconstruction in the lens modelling


# Sersic parameters in the initial simulation
kwargs_sersic = {'I0_sersic': 40, 'R_sersic': 0.015, 'n_sersic': 1, 'q': .8, 'phi_G': 0.5, 'center_x': 0., 'center_y': 0}
if source_type == 'SERSIC':
    source_model_list = ['SERSIC_ELLIPSE']
    kwargs_source_list = [kwargs_sersic]
elif source_type == 'SHAPELETS':
    source_model_list = ['SHAPELETS']
    kwargs_source_list = [kwargs_shapelet]
else:
    raise ValueError("Not valid source_type variable!")

# for this example, we ignore the presence of deflector light
lens_light_model_list = ['NONE']
kwargs_lens_light_list = [{}]


In [ ]:
# initial input simulation

# generate the coordinate grid and image properties
kwargs_data_single = SimAPI.data_configure(numPix, deltaPix, exp_time, sigma_bkg)
# generate the psf variables
kwargs_psf_single = SimAPI.psf_configure(psf_type=psf_type, fwhm=fwhm, kernelsize=11, deltaPix=deltaPix, truncate=3, kernel=None)

# the lens model is a supperposition of an elliptical lens model with external shear
lens_model_list = ['SPEMD', 'SHEAR']
kwargs_lens_list = [kwargs_spemd, kwargs_shear]

kwargs_options = {'lens_model_list': lens_model_list, 
                 'lens_light_model_list': lens_light_model_list,
                 'source_light_model_list': source_model_list,
                 'foreground_shear': True,  # we explicitly model the non-linear shear components
                  'subgrid_res': 1,  # numerical precision in the ray-tracing. Possibility of sub-pixel precision exists
                 }

# generate image
image_real_lensed_shear = SimAPI.im_sim(kwargs_options, kwargs_data_single, kwargs_psf_single, kwargs_lens_list, kwargs_source_list, kwargs_lens_light_list, kwargs_foreground)
# turn 2d data in a 1d data vector
kwargs_data_single['image_data'] = image_real_lensed_shear

In [ ]:
# display the initial simulated image
if plotting:

    cmap_string = 'gray'
    cmap = plt.get_cmap(cmap_string)
    cmap.set_bad(color='k', alpha=1.)
    cmap.set_under('k')

    v_min = -4
    v_max = 1

    f, axes = plt.subplots(1, 1, figsize=(6, 6), sharex=False, sharey=False)

    # sequence of weak lensing
    ax = axes
    im = ax.matshow(np.log10(image_real_lensed_shear), origin='lower', vmin=v_min, vmax=v_max, cmap=cmap, extent=[0, 1, 0, 1])
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
    ax.autoscale(False)

    plt.show()

In [ ]:
# lens models
lens_model_list = []
fixed_lens = []
kwargs_lens_init = []
kwargs_lens_sigma = []
kwargs_lower_lens = []
kwargs_upper_lens = []


fixed_lens.append({})  # for this example, we fix the power-law index of the lens model to be isothermal
kwargs_lens_init.append(kwargs_spemd)
kwargs_lens_sigma.append({'theta_E_sigma': .0001, 'ellipse_sigma':0.005, 'gamma_sigma': 0.1
                    , 'center_x_sigma': 0.0001, 'center_y_sigma': 0.0001})
kwargs_lower_lens.append({'theta_E': 0.01,'q': .5, 'gamma': 1.5, 'phi_G': 0., 'center_x': -10, 'center_y': -10})
kwargs_upper_lens.append({'theta_E': 10,'q': .5, 'gamma': 2.5, 'phi_G': 0., 'center_x': 10, 'center_y': 10})


fixed_lens.append({})
kwargs_lens_init.append(kwargs_shear)
kwargs_lens_sigma.append({'shear_sigma': 0.003})
kwargs_lower_lens.append({'e1': -0.2, 'e2': -0.2})
kwargs_upper_lens.append({'e1': 0.2, 'e2': 0.2})


# lens light models
fixed_lens_light = [{}]
kwargs_lens_light_init = [{}]
kwargs_lens_light_sigma = [{}]
kwargs_lower_lens_light = [{}]
kwargs_upper_lens_light = [{}]


fixed_source = []
kwargs_source_init = []
kwargs_source_sigma = []
kwargs_lower_source = []
kwargs_upper_source = []

for i, model in enumerate(source_model_list):
    if model == 'SHAPELETS':
        fixed_source.append({'n_max': n_max_sim})
        kwargs_source_init.append(kwargs_shapelet)
        kwargs_source_sigma.append({'center_x_sigma': 0.001, 'center_y_sigma': 0.001, 'beta_sigma': shapelet_beta/100., 'n_max_sigma': 2})
        kwargs_lower_source.append({'center_x': -10, 'center_y': -10, 'beta': 0.001, 'n_max': 0})
        kwargs_upper_source.append({'center_x': 10, 'center_y': 10, 'beta': 10, 'n_max': 55})
    if model == 'SERSIC_ELLIPSE':
        fixed_source.append({})
        kwargs_source_init.append(kwargs_sersic)
        kwargs_source_sigma.append({'n_sersic_sigma': 0.001, 'R_sersic_sigma': 0.001, 'ellipse_sigma': 0.001, 'center_x_sigma': 0.0001, 'center_y_sigma': 0.0001})
        kwargs_lower_source.append({'q': .5, 'phi_G': 0, 'R_sersic': 0.001, 'n_sersic': .5, 'center_x': -10, 'center_y': -10})
        kwargs_upper_source.append({'q': .5, 'phi_G': 0, 'R_sersic': 10, 'n_sersic': 5., 'center_x': 10, 'center_y': 10})

fixed_else = {}
kwargs_else_init = kwargs_foreground
kwargs_else_sigma = {'shear_foreground_sigma': 0.001}
kwargs_lower_else = {'gamma1_foreground': -0.2, 'gamma2_foreground': -0.2}
kwargs_upper_else = {'gamma1_foreground': 0.2, 'gamma2_foreground': 0.2}

kwargs_fixed = [fixed_lens, fixed_source, fixed_lens_light, fixed_else]
kwargs_init = [kwargs_lens_init, kwargs_source_init, kwargs_lens_light_init, kwargs_else_init]
kwargs_sigma = [kwargs_lens_sigma, kwargs_source_sigma, kwargs_lens_light_sigma, kwargs_else_sigma]
kwargs_lower = [kwargs_lower_lens, kwargs_lower_source, kwargs_lower_lens_light, kwargs_lower_else]
kwargs_upper = [kwargs_upper_lens, kwargs_upper_source, kwargs_upper_lens_light, kwargs_upper_else]




In [ ]:
from lenstronomy.Workflow.fitting_sequence import FittingSequence

mpi = False  # MPI possible, but not supported through that notebook.
    
job_name = 'shear_test'
input_temp = job_name +'.txt'
output_temp = job_name +'_out.txt'
dir_path = os.getcwd()
path2input_temp = os.path.join(dir_path, input_temp)

fitting_kwargs_list = [{'fitting_routine': 'MCMC', 'n_burn': 100, 'n_run': 100, 'walkerRatio': 10, 'mpi': mpi, 'sigma_scale': .1}]


kwargs_data = [kwargs_data_single]
kwargs_psf = [kwargs_psf_single]
init_samples = None
#f = open(path2input_temp,'wb')
#pickle.dump([kwargs_data, kwargs_psf, kwargs_options, kwargs_init, kwargs_sigma, kwargs_fixed, fitting_kwargs_list, init_samples], f)
#f.close()


start_time = time.time()
fitting_seq = FittingSequence(kwargs_data, kwargs_psf, kwargs_options, kwargs_init, kwargs_sigma, kwargs_fixed, kwargs_lower, kwargs_upper)
lens_result, source_result, lens_light_result, else_result, chain_list, param_list, samples_mcmc, param_mcmc, dist_mcmc = fitting_seq.fit_sequence(fitting_kwargs_list)
kwargs_psf_out = fitting_seq.kwargs_psf
output_ = [lens_result, source_result, lens_light_result, else_result, kwargs_psf_out, chain_list, param_list, samples_mcmc, param_mcmc, dist_mcmc]
input_ = [kwargs_data, kwargs_psf, kwargs_options, kwargs_init, kwargs_sigma, kwargs_fixed, kwargs_lower, kwargs_upper, fitting_kwargs_list, init_samples]
path2dump = job_name + '_out.txt'
#f = open(path2dump, 'wb')
#pickle.dump([input_, output_], f)
#f.close()
end_time = time.time()
print(end_time - start_time, 'total time needed for computation')
print('Result saved in: %s' % path2dump)
print('============ CONGRATULATION, YOUR JOB WAS SUCCESSFUL ================ ')

## analyse MCMC chain

In [ ]:
import lenstronomy_extensions.Plots.output_plots as out_plot
# pre-computed MCMC chains:
# job_name_out = 'test': an elliptical Sersic source light profile
# job_name_out = 'shear_test': a shapelet source model based on M31

job_name_out = 'shear_test'
input_temp = job_name_out +'.txt'
output_temp = job_name_out +'_out.txt'

#f = open(output_temp)
#[input_, output_] = pickle.load(f)
#f.close()
kwargs_data, kwargs_psf, kwargs_options, kwargs_init, kwargs_sigma, kwargs_fixed, kwargs_lower, kwargs_upper, fitting_kwargs_list, init_samples = input_
lens_result, source_result, lens_light_result, else_result, kwargs_psf, chain_list, param_list, samples_mcmc, param_mcmc, dist_mcmc = output_

print "Input parameters in the lens model:"
print kwargs_init[0]

band_i = 0
from lenstronomy_extensions.Plots.output_plots import LensModelPlot
lensPlot = LensModelPlot(kwargs_options, kwargs_data[band_i], kwargs_psf[band_i], lens_result, source_result, lens_light_result,
                        else_result, arrow_size=0.02, cmap_string="gist_heat", high_res=5)
    
if plotting:  
    f, axes = plt.subplots(2, 3, figsize=(16, 8), sharex=False, sharey=False)

    lensPlot.data_plot(ax=axes[0,0])
    lensPlot.model_plot(ax=axes[0,1])
    lensPlot.normalized_residual_plot(ax=axes[0,2], v_min=-6, v_max=6)
    lensPlot.source_plot(ax=axes[1, 0],convolution=False, deltaPix_source=0.01, numPix=100)
    lensPlot.convergence_plot(ax=axes[1, 1], v_max=1)
    lensPlot.magnification_plot(ax=axes[1, 2])
    f.tight_layout()
    f.subplots_adjust(left=None, bottom=None, right=None, top=None, wspace=0., hspace=0.05)
    plt.show()


In [ ]:
# the results of the MCMC chain, split in two corner plots

print "number of non-linear parameters in the MCMC process: ", len(param_mcmc)
print "parameters in order: ", param_mcmc
print "number of evaluations in the MCMC process: ", np.shape(samples_mcmc)[0]
if not samples_mcmc == []:
    if plotting:
        n, num_param = np.shape(samples_mcmc)
        plot = corner.corner(samples_mcmc[:,:8], labels=param_mcmc[:8], show_titles=True)
        plot = corner.corner(samples_mcmc[:,8:], labels=param_mcmc[8:], show_titles=True)